In [1]:
import pandas as pd
import os
import re
from tqdm import tqdm
from nltk.tokenize import WhitespaceTokenizer

In [2]:
os.chdir('/Volumes/T7/chroniclingamerica/redsummer-1917-1921')

In [2]:
location_keyword='chicago'
event_keyword='riot'

In [82]:
tk=WhitespaceTokenizer()
page_library=[]
date_library=[]
lccn_library=[]
for i in tqdm(os.listdir()):
    keywords=[location_keyword,event_keyword]
    if i.endswith('.feather'):
        lccn=re.search('^.*-part',i).group(0).replace('-part','')
        df=pd.read_feather(i)
        for element in range(0, len(df['text'])):
            try:
                page=df.text[element].lower()
                if (location_keyword in tk.tokenize(page)) \
                    and (event_keyword in tk.tokenize(page)):
                    page_library.append(page)
                    date_library.append(df.date[element])
                    lccn_library.append(lccn)
            except:
                pass

100%|██████████| 17203/17203 [33:24<00:00,  8.58it/s]  


In [89]:
pd.DataFrame({'lccn':lccn_library,'date':date_library,'text':page_library}).to_feather('../1919_chicago.feather')

visualization on map

In [3]:
chicago=pd.read_feather('../1919_chicago.feather')
ca=pd.read_csv('/Volumes/Macintosh HD/Users/mstudio/repository/violence/chronicling-america-list.txt', sep='|')

In [83]:
fipscode=pd.read_csv('https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt', sep='\t')

In [31]:
ca.columns = ca.columns.str.strip()
ca['lccn']=ca['LCCN']
ca['lccn']=ca['lccn'].str.strip()

In [94]:
chicago_loc=pd.merge(chicago,ca,how='left',on='lccn')
chicago_loc['State']=chicago_loc['State'].str.strip()
chicago_loc['fips']=chicago_loc['Title'].str.extract(r'(\(.*?\))')
chicago_loc['city']=chicago_loc['fips'].str.extract(r'(.*?,)')
chicago_loc['city']=chicago_loc['city'].str.replace('[','')
chicago_loc['city']=chicago_loc['city'].str.replace(']','')
chicago_loc['city']=chicago_loc['city'].str.replace('(','').str.replace(',','').str.strip()


In [104]:
#https://public.opendatasoft.com/explore/dataset/us-county-boundaries/export/?disjunctive.statefp&disjunctive.countyfp&disjunctive.name&disjunctive.namelsad&disjunctive.stusab&disjunctive.state_name
uscities=pd.read_csv('/Volumes/Macintosh HD/Users/mstudio/repository/violence/uscities.csv')
uscities['State']=uscities['state_name']

In [105]:
viz_chi=chicago_loc.merge(uscities,how='left',on=['city','State'])[['lccn','date','city','lat','lng']]
count_dic=pd.DataFrame(viz_chi.groupby('city').count()['lccn']).to_dict()
viz_chi['count']=viz_chi['city'].apply(lambda x: count_dic['lccn'].get(x))

In [142]:
df=viz_chi[~viz_chi['count'].isnull()]

In [153]:
import pandas as pd
import plotly.express as px

fig = px.scatter_mapbox(
    df, lat="lat", lon="lng", hover_name="city", hover_data=["lccn"],
    size=df['count'], color_discrete_sequence=["fuchsia"], zoom=2)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [149]:
fig.write_html("/Volumes/Macintosh HD/users/mstudio/repository/violence/file.html")